In [1]:
import os
from transformers.optimization import get_linear_schedule_with_warmup
from transformers import logging
logging.set_verbosity_warning()
from transformers import AutoTokenizer
from data_utils import *
from model_utils import *
import torch
from torch.utils.data import DataLoader
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
import gc
import numpy as np
gc.disable()
from eval import evaluate
import pdb
import torch.nn.functional as F

/home/ubuntu/miniconda3/envs/mlingual/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
torch.tensor(42).shape == []

False

In [7]:
# ----------------------------------------------------
model_name = 'sentence-transformers/use-cmlm-multilingual'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
txt = ['A man jumped over the bridge', 'A man jumped over the bridge. A man jumped over the bridge. ']
txt_tok = [tokenizer.encode_plus(t, truncation=True, max_length=121, return_tensors='pt') for t in txt]

input_ids = [item['input_ids'].reshape(-1) for item in txt_tok]
input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)

In [9]:
tokenizer.pad_token_id

0

In [10]:
input_ids

tensor([[   101,    138,  15351, 281182,  15444,  14985,  71426,    102,      0,
              0,      0,      0,      0,      0,      0,      0],
        [   101,    138,  15351, 281182,  15444,  14985,  71426,    119,    138,
          15351, 281182,  15444,  14985,  71426,    119,    102]])

In [11]:
pad_sequence?

Signature:
pad_sequence(
    sequences: Union[torch.Tensor, List[torch.Tensor]],
    batch_first: bool = False,
    padding_value: float = 0.0,
) -> torch.Tensor
Docstring:
Pad a list of variable length Tensors with ``padding_value``.

``pad_sequence`` stacks a list of Tensors along a new dimension,
and pads them to equal length. For example, if the input is a list of
sequences with size ``L x *`` and ``batch_first`` is False, the output is
of size ``T x B x *``.

`B` is batch size. It is equal to the number of elements in ``sequences``.
`T` is length of the longest sequence.
`L` is length of the sequence.
`*` is any number of trailing dimensions, including none.

Example:
    >>> from torch.nn.utils.rnn import pad_sequence
    >>> a = torch.ones(25, 300)
    >>> b = torch.ones(22, 300)
    >>> c = torch.ones(15, 300)
    >>> pad_sequence([a, b, c]).size()
    torch.Size([25, 3, 300])

Note:
    This function returns a Tensor of size ``T x B x *`` or ``B x T x *``
    where `T` is the 

In [12]:
# ----------------------------------------------------
# global variable declaration
# ----------------------------------------------------
model_name = 'sentence-transformers/use-cmlm-multilingual'
external_dim1 = 1536 
external_dim2 = 3072
device = 'cuda:0'
scale = 20
batch_size = 32
max_seq_len = 312
eval_data_root = './../sample_data'
train_data_root = './../in_data'
epochs = 2

In [13]:
from model_utils import FeatureSet, AggregationMethod, ModifiedSentenceTransformer
# ----------------------------------------------------
# Model declaration
# ----------------------------------------------------
model = model_attention = ModifiedSentenceTransformer(model_name=model_name, 
                                                      external_dim1=external_dim1, 
                                                      external_dim2=external_dim2, 
                                                      aggregation_method=AggregationMethod.LINEAR,
                                                      feature_set=FeatureSet.SBERT_EXT1_EXT2)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at sentence-transformers/use-cmlm-multilingual were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: 

In [ ]:
# ----------------------------------------------------
# Dataset & Dataloader declaration
# ----------------------------------------------------
# Get and Process the raw task dataset
eval_facts, eval_posts, eval_mapping = get_raw_data_df(fact_path=os.path.join(eval_data_root, "trial_fact_checks.csv"), 
                                                posts_path=os.path.join(eval_data_root, "trial_posts.csv"),
                                                post2fact_mapping_path=os.path.join(eval_data_root, "trial_data_mapping.csv"))
eval_facts = process_facts_df(eval_facts)
eval_posts = process_posts_df(eval_posts)

# train_facts, train_posts, train_mapping = get_raw_data_df(fact_path=os.path.join(train_data_root, "fact_checks.csv"), 
#                                                     posts_path=os.path.join(train_data_root, "posts.csv"),
#                                                     post2fact_mapping_path=os.path.join(train_data_root, "fact_check_post_mapping.csv"))   
# train_facts = process_facts_df(train_facts)
# train_posts = process_posts_df(train_posts)

In [ ]:
result = evaluate(model=model, tokenizer=tokenizer, device=device, 
                  eval_posts=eval_posts, eval_facts=eval_facts, eval_mapping=eval_mapping, 
                  fact_orig_emb_pth = './../openai-op/eval_orig-fact.pkl', fact_eng_emb_pth = './../openai-op/eval_eng-fact.pkl', 
                  post_l1_emb_pth = './../openai-op/eval_l1-post.pkl', post_l2_emb_pth = './../openai-op/eval_l2-post.pkl',
                  batch_size = 32, top_K = 5)

100%|██████████| 2/2 [00:00<00:00,  3.24it/s]


In [ ]:
# Create the list of anchor-positive_example pair list for the training data
eval_pair_dataset, eval_ext_emb_small, eval_ext_emb_large = create_train_dataset(df_facts = eval_facts, df_posts = eval_posts, df_mapping = eval_mapping, 
                                                                                            fact_orig_emb_pth = './../openai-op/eval_orig-fact.pkl', 
                                                                                            fact_eng_emb_pth = './../openai-op/eval_eng-fact.pkl', 
                                                                                            post_l1_emb_pth = './../openai-op/eval_l1-post.pkl', 
                                                                                            post_l2_emb_pth = './../openai-op/eval_l2-post.pkl')

In [ ]:
# temp_df = create_post_fact_df_with_ext_emb(df_facts= eval_facts, df_posts= eval_posts, df_mapping= eval_mapping, 
#                                             fact_orig_emb_pth = './../openai-op/eval_orig-fact.pkl', 
#                                             fact_eng_emb_pth = './../openai-op/eval_eng-fact.pkl', 
#                                             post_l1_emb_pth = './../openai-op/eval_l1-post.pkl', 
#                                             post_l2_emb_pth = './../openai-op/eval_l2-post.pkl',
#                                             strategy=MergeStrategy.ALL)

In [ ]:
def evaluate(eval_posts, eval_facts, fact_orig_emb_pth, fact_eng_emb_pth, post_l1_emb_pth, post_l2_emb_pth):

    fact_ix2id_mapping = {ix: row['fact_check_id'] for ix, row in eval_facts.iterrows()}
    get_fact_id = lambda x: fact_ix2id_mapping[x]
    vfunc = np.vectorize(get_fact_id)

    eval_dict = {'post_l1': TextType.POST_L1, 'post_l2': TextType.POST_L2, 
                'fact_orig': TextType.FACT_ORIG, 'fact_eng': TextType.FACT_ENG}
    eval_data_inf = {}
    eval_dataset_inf = {}
    eval_dl_inf = {}
    eval_model_repr = {}
    temp_df = cc(eval_mapping)

    for key, v in eval_dict.items():
        text_list, small_emb_list, large_emb_list = create_inference_dataset(df_facts=eval_facts, df_posts=eval_posts, df_mapping=eval_mapping, 
                                                                            fact_orig_emb_pth = './../openai-op/eval_orig-fact.pkl', 
                                                                            fact_eng_emb_pth = './../openai-op/eval_eng-fact.pkl', 
                                                                            post_l1_emb_pth = './../openai-op/eval_l1-post.pkl', 
                                                                            post_l2_emb_pth = './../openai-op/eval_l2-post.pkl',
                                                                            txt_type=v)
        eval_data_inf[key] = (text_list, small_emb_list, large_emb_list)
        eval_dataset_inf[key] = PairDatasetWithEmbeddings_for_inf(txt_list=text_list, external_embeddings1=small_emb_list, 
                                                                external_embeddings2=large_emb_list, tokenizer=tokenizer)
        eval_dl_inf[key] = DataLoader(eval_dataset_inf[key], batch_size=32, shuffle=False)
        eval_model_repr[key] = []

        # Get model representation of the posts (l1, l2) and facts (orig, eng)
        model.eval()
        with torch.no_grad():
            for ix, batch in tqdm(enumerate(eval_dl_inf[key])):
                batch = {k: v.to(device) for k, v in batch.items()}
                res = model(batch)
                res = res.cpu().detach().numpy()
                eval_model_repr[key].extend(res)
            
            if 'post' in key:
                df = cc(eval_posts)
                df[f'pred_{key}'] = eval_model_repr[key]
                relevant_df = df[['post_id', f'pred_{key}']]
                temp_df = temp_df.merge(relevant_df, how = 'left')

            if 'fact' in key:
                df = cc(eval_facts)
                df[f'pred_{key}'] = eval_model_repr[key]
                relevant_df = df[['fact_check_id', f'pred_{key}']]
                temp_df = temp_df.merge(relevant_df, how = 'left')

In [ ]:
fact_ix2id_mapping = {ix: row['fact_check_id'] for ix, row in eval_facts.iterrows()}
get_fact_id = lambda x: fact_ix2id_mapping[x]
vfunc = np.vectorize(get_fact_id)

In [ ]:
eval_dict = {'post_l1': TextType.POST_L1, 'post_l2': TextType.POST_L2, 
             'fact_orig': TextType.FACT_ORIG, 'fact_eng': TextType.FACT_ENG}

eval_data_inf = {}
eval_dataset_inf = {}
eval_dl_inf = {}
eval_model_repr = {}
temp_df = cc(eval_mapping)

for key, v in eval_dict.items():
    text_list, small_emb_list, large_emb_list = create_inference_dataset(df_facts=eval_facts, df_posts=eval_posts, df_mapping=eval_mapping, 
                                                                         fact_orig_emb_pth = './../openai-op/eval_orig-fact.pkl', 
                                                                         fact_eng_emb_pth = './../openai-op/eval_eng-fact.pkl', 
                                                                         post_l1_emb_pth = './../openai-op/eval_l1-post.pkl', 
                                                                         post_l2_emb_pth = './../openai-op/eval_l2-post.pkl',
                                                                         txt_type=v)
    eval_data_inf[key] = (text_list, small_emb_list, large_emb_list)
    eval_dataset_inf[key] = PairDatasetWithEmbeddings_for_inf(txt_list=text_list, external_embeddings1=small_emb_list, 
                                                            external_embeddings2=large_emb_list, tokenizer=tokenizer)
    eval_dl_inf[key] = DataLoader(eval_dataset_inf[key], batch_size=32, shuffle=False)
    eval_model_repr[key] = []

    # Get model representation of the posts (l1, l2) and facts (orig, eng)
    model.eval()
    with torch.no_grad():
        for ix, batch in tqdm(enumerate(eval_dl_inf[key])):
            batch = {k: v.to(device) for k, v in batch.items()}
            res = model(batch)
            res = res.cpu().detach().numpy()
            eval_model_repr[key].extend(res)
          
        if 'post' in key:
            df = cc(eval_posts)
            df[f'pred_{key}'] = eval_model_repr[key]
            relevant_df = df[['post_id', f'pred_{key}']]
            temp_df = temp_df.merge(relevant_df, how = 'left')

        if 'fact' in key:
            df = cc(eval_facts)
            df[f'pred_{key}'] = eval_model_repr[key]
            relevant_df = df[['fact_check_id', f'pred_{key}']]
            temp_df = temp_df.merge(relevant_df, how = 'left')

2it [00:00,  5.18it/s]
2it [00:00,  5.19it/s]
2it [00:00,  4.89it/s]
2it [00:00,  4.89it/s]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
metric_keys = [('post_l1', 'fact_eng'), ('post_l1', 'fact_orig'), ('post_l2', 'fact_eng'), ('post_l2', 'fact_orig')]

for post_key, fact_key in metric_keys:
    print(f'{post_key} <-> {fact_key}')
    pred = cosine_similarity(temp_df[f'pred_{post_key}'].tolist(), temp_df[f'pred_{fact_key}'].tolist())

    res = []
    for k in range(1, 6):
        preds = (-pred).argsort()[:, :k]
        res.append(np.mean([(i in pred) for  i, pred in enumerate(preds)]))
    print(f'{res}\n')

post_l1 <-> fact_eng
[0.62, 0.8, 0.82, 0.84, 0.84]

post_l1 <-> fact_orig
[0.64, 0.74, 0.84, 0.84, 0.86]

post_l2 <-> fact_eng
[0.7, 0.82, 0.86, 0.9, 0.92]

post_l2 <-> fact_orig
[0.68, 0.76, 0.84, 0.84, 0.86]



In [ ]:
def get_top_k_acc(pred_fact_id_matrix, true_fact_id_list, top_K):
    result = []

    for true_id, pred_ids in zip(true_fact_id_list, pred_fact_id_matrix):
        correct_pred = [(true_id in pred_ids[:k]) for k in range(1, (top_K + 1))]
        result.append(correct_pred)
        
    result = np.array(result).mean(axis = 0)
    return result

In [ ]:
eval_results = {}
metric_keys = [('post_l1', 'fact_eng'), ('post_l1', 'fact_orig'), ('post_l2', 'fact_eng'), ('post_l2', 'fact_orig')]

for post_key, fact_key in metric_keys:
    print(f'{post_key} <-> {fact_key}')

    fact_repr_list = eval_model_repr[fact_key]    
    post_repr_list = temp_df[f'pred_{post_key}'].tolist()
    true_fact_id_list = temp_df[f'fact_check_id'].tolist()

    pred = cosine_similarity(post_repr_list, fact_repr_list)
    pred_fact_ids = vfunc((-pred).argsort())
    
    res = get_top_k_acc(pred_fact_ids, true_fact_id_list, top_K = 5)
    eval_results[f'{post_key}_{fact_key}'] = res

post_l1 <-> fact_eng
[0.62 0.8  0.82 0.84 0.84]
post_l1 <-> fact_orig
[0.64 0.74 0.84 0.84 0.86]
post_l2 <-> fact_eng
[0.7  0.82 0.86 0.9  0.92]
post_l2 <-> fact_orig
[0.68 0.76 0.84 0.84 0.86]


In [ ]:
pred = cosine_similarity(temp_df[f'pred_{post_key}'].tolist(), eval_model_repr[fact_key])

In [ ]:
fact_ix2id_mapping = {ix: row['fact_check_id'] for ix, row in eval_facts.iterrows()}
get_post_id = lambda x: fact_ix2id_mapping[x]
vfunc = np.vectorize(get_post_id)

preds = vfunc((-pred).argsort())

In [ ]:
pred_fact_ids

array([[27, 28, 33, ..., 46, 14,  1],
       [19, 11, 16, ...,  1, 26, 27],
       [ 9,  5, 37, ..., 48, 29, 45],
       ...,
       [26, 18, 41, ...,  2,  9,  1],
       [ 1, 41,  9, ..., 29, 27, 30],
       [ 6, 25,  4, ..., 22, 45, 21]])

In [ ]:
correct = []
for true, pred in zip(temp_df['fact_check_id'], preds):
    correct.append(true in pred[:5])
np.mean(correct)

0.84